In [1]:
import pandas as pd 

In [2]:
patients = pd.read_csv(r'C:\Users\yavuzcevik\Desktop\hospital\patients.csv')
services_weekly = pd.read_csv(r'C:\Users\yavuzcevik\Desktop\hospital\services_weekly.csv')
staff = pd.read_csv(r'C:\Users\yavuzcevik\Desktop\hospital\staff.csv')
staff_schedule= pd.read_csv(r'C:\Users\yavuzcevik\Desktop\hospital\staff_schedule.csv')

<h2>Tabloları ilk etapta  null veriler ve boş satırlar varmı diye  kontrol ettim  daha sonra duplicated() fonksiyonu ile tekrar eden değerler varmı diye baktım, problemin sadece tarih sütunlarının veri tiplerinde olduğunu gördüm ve onları düzelttim</h2>

In [3]:
patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   patient_id      1000 non-null   object
 1   name            1000 non-null   object
 2   age             1000 non-null   int64 
 3   arrival_date    1000 non-null   object
 4   departure_date  1000 non-null   object
 5   service         1000 non-null   object
 6   satisfaction    1000 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 54.8+ KB


In [4]:
patients['arrival_date'] = pd.to_datetime(patients['arrival_date']) 
patients['departure_date'] = pd.to_datetime(patients['departure_date']) 

In [5]:
services_weekly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   week                  208 non-null    int64 
 1   month                 208 non-null    int64 
 2   service               208 non-null    object
 3   available_beds        208 non-null    int64 
 4   patients_request      208 non-null    int64 
 5   patients_admitted     208 non-null    int64 
 6   patients_refused      208 non-null    int64 
 7   patient_satisfaction  208 non-null    int64 
 8   staff_morale          208 non-null    int64 
 9   event                 208 non-null    object
dtypes: int64(8), object(2)
memory usage: 16.4+ KB


In [6]:
staff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   staff_id    110 non-null    object
 1   staff_name  110 non-null    object
 2   role        110 non-null    object
 3   service     110 non-null    object
dtypes: object(4)
memory usage: 3.6+ KB


In [7]:
staff_schedule.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6552 entries, 0 to 6551
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   week        6552 non-null   int64 
 1   staff_id    6552 non-null   object
 2   staff_name  6552 non-null   object
 3   role        6552 non-null   object
 4   service     6552 non-null   object
 5   present     6552 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 307.3+ KB


In [8]:
for df in [patients, services_weekly, staff, staff_schedule]:
     print(df.duplicated().sum())

0
0
0
0


<h1>ANALİZ AŞAMASI</h1>

<h3>Her hasta için hastanede kalış süresini hesaplayan bir kolon açtım</h3>

In [9]:
patients['stay_duration_days'] = (patients['departure_date'] - patients['arrival_date']).dt.days
patients.head()

,patient_id,name,age,arrival_date,departure_date,service,satisfaction,stay_duration_days
0,PAT-09484753,Richard Rodriguez,24,2025-03-16,2025-03-22,surgery,61,6
1,PAT-f0644084,Shannon Walker,6,2025-12-13,2025-12-14,surgery,83,1
2,PAT-ac6162e4,Julia Torres,24,2025-06-29,2025-07-05,general_medicine,83,6
3,PAT-3dda2bb5,Crystal Johnson,32,2025-10-12,2025-10-23,emergency,81,11
4,PAT-08591375,Garrett Lin,25,2025-02-18,2025-02-25,ICU,76,7


<h3>Hastaların ortalama yatış günü</h3>

In [10]:
ortalama_yatis_gün = patients['stay_duration_days'].mean()
print('Ortalama Yatış Günü :', ortalama_yatis_gün)

Ortalama Yatış Günü : 7.407


<h3>Hastaların servise göre yattığı gün ortalaması</h3>

In [11]:
servis_bazinda_ortalama =  patients.groupby('service')['stay_duration_days'].mean()
print(servis_bazinda_ortalama)

service
ICU                 7.605809
emergency           7.159696
general_medicine    6.995868
surgery             7.866142
Name: stay_duration_days, dtype: float64


<h3>Hastaların ortalama yaşı</h3>

In [12]:
ortalama_yas =  patients['age'].mean()
print('Hastaların ortalama yaşı: ', ortalama_yas)

Hastaların ortalama yaşı:  45.337


<h3>Hastaların memnuniyet için verdikleri sayıları 3 kategoride sınıflandırıp tanımladım  ve bunları yeni bir sütun olarak tablomuza ekliyorum </h3>

In [13]:

bins = [0, 50, 70, 100] 
labels = ['(Düşük) Memnuniyet', '(Orta) Memnuniyet', '(Yüksek) Memnuniyet']

patients['satisfaction_group'] = pd.cut(
    patients['satisfaction'], 
    bins=bins, 
    labels=labels, 
    include_lowest=True
)

print(patients['satisfaction_group'].value_counts())
patients.head()

satisfaction_group
(Yüksek) Memnuniyet    738
(Orta) Memnuniyet      262
(Düşük) Memnuniyet       0
Name: count, dtype: int64


,patient_id,name,age,arrival_date,departure_date,service,satisfaction,stay_duration_days,satisfaction_group
0,PAT-09484753,Richard Rodriguez,24,2025-03-16,2025-03-22,surgery,61,6,(Orta) Memnuniyet
1,PAT-f0644084,Shannon Walker,6,2025-12-13,2025-12-14,surgery,83,1,(Yüksek) Memnuniyet
2,PAT-ac6162e4,Julia Torres,24,2025-06-29,2025-07-05,general_medicine,83,6,(Yüksek) Memnuniyet
3,PAT-3dda2bb5,Crystal Johnson,32,2025-10-12,2025-10-23,emergency,81,11,(Yüksek) Memnuniyet
4,PAT-08591375,Garrett Lin,25,2025-02-18,2025-02-25,ICU,76,7,(Yüksek) Memnuniyet


In [14]:
services_weekly.head()

,week,month,service,available_beds,patients_request,patients_admitted,patients_refused,patient_satisfaction,staff_morale,event
0,1,1,emergency,32,76,32,44,67,70,none
1,1,1,surgery,45,130,45,85,83,78,flu
2,1,1,general_medicine,37,201,37,164,97,43,flu
3,1,1,ICU,22,31,22,9,84,91,flu
4,2,1,emergency,28,169,28,141,75,64,none


<h3>Services_weekly tablosunu  servislere göre gruplandırarak hastanenin hangi bölümüne ne kadar 'hasta yatış talebi'  gelmiş bunu buluyoruz</h3>

In [15]:
toplam_talep = services_weekly.groupby('service')['patients_request'].sum()
print('Toplam Hasta Talebi Servis Bazında :', toplam_talep)




Toplam Hasta Talebi Servis Bazında : service
ICU                  789
emergency           6193
general_medicine    4270
surgery             2241
Name: patients_request, dtype: int64


<h3>Services_weekly  tablosunda  hastaların yatış taleplerinin kabul edilme oranını inceliyorum.
  -   ICU vs surgery  bölümleri  diğer bölümlere göre bariz daha başarılıdır yatak kapasitesi ve gelen talep kabul oranıyla bunu görebiliyoruz
  -   emergency bölümü ise kriz içerisindedir diyebilirim  yüksek talep olmasına rağmen yatak kapasitesi talebe göre çok düşük kalmış ve bu yüzden çoğu hastanın talebi red edilmiş</h3>

In [16]:

kabul_oranlari = services_weekly.groupby('service').agg(
    total_admitted=('patients_admitted', 'sum'),
    total_request=('patients_request', 'sum')
)

kabul_oranlari['acceptance_rate'] = (
    kabul_oranlari['total_admitted'] / kabul_oranlari['total_request']
) * 100


print("--- Servis Bazında Yatış Kabul Oranı (%) ---\n")
print(kabul_oranlari.sort_values(by='acceptance_rate', ascending=False).round(2))

--- Servis Bazında Yatış Kabul Oranı (%) ---

                  total_admitted  total_request  acceptance_rate
service                                                         
ICU                          648            789            82.13
surgery                     1686           2241            75.23
general_medicine            2332           4270            54.61
emergency                   1185           6193            19.13


<h3>Hastanedeki doktor, hemşire ve  asistan hemşirelerin sayısı hesaplıyoruz</h3>

In [26]:
staff.groupby('role')['staff_name'].count()
    


role
doctor               18
nurse                69
nursing_assistant    23
Name: staff_name, dtype: int64

<h3>Doktorların kaç hafta çalıştıklarını hesaplıyorum  </h3>

In [32]:
doctors = staff_schedule[staff_schedule['role']== 'doctor']
most_active = doctors.groupby('staff_name')['present'].sum().sort_values(ascending = False )

print(most_active)

staff_name
Abigail Shaffer      33
Brittany Farmer      33
Connie Lawrence      33
Carol Tucker         32
Daniel Wagner        32
Anna Henderson       32
John Whitehead       32
Paula Moreno         32
Jessica Callahan     32
Aaron Wise           31
Fred Smith           31
Deborah Figueroa     31
Allison Hill         30
Tracy House          30
Gina Moore           30
Denise Jacobs        30
Cristian Santos      30
Angie Henderson      29
James Brown          29
Christina Walters    29
Amber Kidd           28
Noah Rhodes          28
Name: present, dtype: int64
